# Predicting Gibbs free energy of metabolic reactions using Neural Network
Motivation: The free energy of a biochemical reaction can be calculated from the following equation:

$$\Delta G_{rxn} = S^T\Delta G^\circ_f + RT\cdot S^T\log c$$
where $S$ is the stoichiometric matrix of reactions and metabolites, $\Delta G^\circ_f$ is the standard free energy of formation of the reactants and products, $R$ is the gas constant, $T$ is the temperature and $\log c$ is the log concentrations of the reactants and products.
Because $\Delta G_{rxn}$ is proportional to the log concentrations, the standard free energy of formation for the products and reactants is the dominating source of error. 

Current methods to estimate $\Delta G^\circ_f$ are based on empirical methods such as component contribution and group contribution. These methods are fast, but inaccurate.  More accurate methods involve quantum chemistry, but are very expensive to compute.  Recently a family of deep learning methods colletively known as message passwing neural networks have shown to be both a fast and accurate method for computing molecular properties, including $\Delta G^\circ_f$ when trained on a large corpus of molecular properties that have been pre-computed using quantum chemistry methods.



## Metrics: 



1. Our goal is to predict the free energy of metabolites (unknown/not trained) involved in metacyc within the accuracy of DFT by training quantum mechanical data (QM9).
2. Generate Gibbs energy of metabolic reactions (Need to think about how can we do this)
3.  If the machine learning predicted free energy of metabolic reactions is within 1-2 kcal/mol than that of experimental value, we are successful. 


## Deep Learning Approaches
Supervised learning on molecules has seen rapid improvements with applications to chemistry, drug discovery, and materials science
* Message-Passing Neural Networks ([Gilmer et al 2017](https://arxiv.org/pdf/1704.01212.pdf)) A framework for describing many graph neural networks (described below) in terms of  Message, Update and  Readout operations on graphs with analogy to message passing in Probabilistic Graphical models
  * Graph Convolutional Networks  ([Thomas Kipf et al 2016](http://arxiv.org/abs/1609.02907))
  * Gated Graph Convolutional Networks [(Li et al 2016)](http://arxiv.org/abs/1511.05493)
  * [Interaction Networks](https://github.com/PNNL-CompBio/graph-neural-networks) ([Battaglia et al 2016](http://arxiv.org/abs/1612.00222)) This is the network we got working first
  * [Deep Tensor Neural Networks](https://github.com/atomistic-machine-learning/dtnn) [(Schutt 2017a)](https://www.nature.com/articles/ncomms13890) This was referenced in MPNN paper
  * [SchNet](https://github.com/djinnome/SchNet) ([Schutt 2017b](http://arxiv.org/abs/1712.06113)) This improved on DTNN
  * Neural Message Passing with Edge Updates [(Bjorgensen 2018)](https://arxiv.org/pdf/1806.03146.pdf) This improved on SchNet.
  * Graph Networks[(Battaglia et al 2018)](https://arxiv.org/abs/1806.01261) This is a generalization of MPNNs (see figure below)
* Ensemble networks to predict properties. According to the MPNN paper: 
      We were able to further improve performance on the test set by ensembling the predictions of the five models with lowest validation error.
    


![graph neural network architectures  Figure 4 from (Battagila et al 2018)](https://ndownloader.figshare.com/files/12245093/preview/12245093/preview.jpg?private_link=7bc3719fab09b0639bd4)

## Training Data (110K molecules, 13 properties)
* Graph: Quantum mechanics data (QM9/B3LYP) raw distances and chemical graph 
* Labels: 13 properties

    
## Validation and dev data (20K molecules, 13 properties)
* Graph: Quantum mechanics data (QM9/B3LYP) raw distances and chemical graph
* Labels: 13 properties

## Test data (400 molecules, 13 properties)
* MetaCyc subset that is in QM9
* Equilibrator subset that is in QM9

## Molecule Prediction data (3K molecules, 1 property $\Delta G^\circ_{f}$)
* MetaCyc (Group Contribution) subset that is not in the QM9 data and has <= 9 carbons and only consists of (C H N O F) atoms
* Equilibrator (Component Contribution) subset that is not in the QM9 data

## Reaction Prediction data (1200 reactions, 1 property $\Delta G^\circ_{rxn}$)
* Metacyc reactions consisting of only QM9-friendly substrates.

## Experimental (Golden set) data (~100 reactions)
* NIST free energy of reaction for quantification of true error in prediction

## Preliminary results

Last night, we submitted  Interaction Networks architecture (Battaglia 2016) for training on QM9 for raw distance

After analyzing the trained network, we will predict Gibbs energy from our Dev set.


## Challenges
Rep. of input molecules for prediction of properties: There are a number of things we need to modify in the Message Passing Neural Network (MPNN) in order to predict free energy of reactions involved in metabolism

1. 3D Coordinates: We dont have accurate 3D coordinates of the lowest energy states for MetaCyc/Equilibrator compounds. We know that MPNN gets 11/13 properties within chemical accuracy of DFT using both 3D coordinates of the lowest energy state and the SMILES, whereas it  gets 5/13 properties accurately from the Inchi/Smiles only.

2. Free Energy of formation to free energy of reaction: 
We just have values for the gibbs free energy of formation $\Delta G^\circ_{f}$ and from that we can get free energy of reaction $\Delta G^\circ_{rxn}$ (by calculating $\Delta G_{rxn}^\circ = S^T\Delta G^\circ_{f}$) where $S$ is the stoichiometric matrix of reactions and metabolites.  This will enable us to predict a more accurate free energy of reaction from the QM9 data. The challenge will be how to correctly propagate errors from the Gibbs free energy of formation to the Gibbs free energy of reaction.  We also need to find a data set that contains experimentally measured Gibbs free energies of reactions as a (NIST?)

3. The QM9 dataset contains only molecules with atoms in C, H, N, O, and F.  It hasn't been trained on any molecules with P or S, or with any cofactors. It has also only been trained on molecules with 9 or fewer carbons, so we should not expect to get reasonable results with larger molecules or phosphorylated molecules

We need to extend to multiple NN architectures (try SchNet  and SchNet + edge updates) 


In [1]:
def qm9_edges(g, e_representation='raw_distance'):
    remove_edges = []
    e={}    
    for n1, n2, d in g.edges_iter(data=True):
        e_t = []
        # Raw distance function
        if e_representation == 'chem_graph':
            if d['b_type'] is None:
                remove_edges += [(n1, n2)]
            else:
                e_t += [i+1 for i, x in enumerate([rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC])
                        if x == d['b_type']]
        elif e_representation == 'distance_bin':
            if d['b_type'] is None:
                step = (6-2)/8.0
                start = 2
                b = 9
                for i in range(0, 9):
                    if d['distance'] < (start+i*step):
                        b = i
                        break
                e_t.append(b+5)
            else:
                e_t += [i+1 for i, x in enumerate([rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                   rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC])
                        if x == d['b_type']]
        elif e_representation == 'raw_distance':
            if d['b_type'] is None:
                remove_edges += [(n1, n2)]
            else:
                e_t.append(d['distance'])
                e_t += [int(d['b_type'] == x) for x in [rdkit.Chem.rdchem.BondType.SINGLE, rdkit.Chem.rdchem.BondType.DOUBLE,
                                                        rdkit.Chem.rdchem.BondType.TRIPLE, rdkit.Chem.rdchem.BondType.AROMATIC]]
        else:
            print('Incorrect Edge representation transform')
            quit()
        if e_t:
            e[(n1, n2)] = e_t
    for edg in remove_edges:
        g.remove_edge(*edg)
    return nx.to_numpy_matrix(g), e


## Future ROAD MAP

By developing deep neural network we would like to expand the range of problems that can be addressed by 
1. Accurately modeling chemical properties with high level ab iniito simulations
2. Modeling much larger systems including cofactors and transition metal
3. Trainlarger datasets, and 